In [1]:
import pandas as pd
data = pd.read_csv('training_data.csv')
data.describe()
test_well = data[data['Well Name'] == 'SHANKLE']
data = data[data['Well Name'] != 'SHANKLE']
#test_well = data[data['Well Name'] == 'SHRIMPLIN']
#data = data[data['Well Name'] != 'SHRIMPLIN']

In [2]:
features = ['GR', 'ILD_log10', 'DeltaPHI','PHIND','PE','NM_M', 'RELPOS']
feature_vectors = data[features]
facies_labels = data['Facies']
facies_labels.describe()

count    2783.000000
mean        4.558390
std         2.515249
min         1.000000
25%         2.000000
50%         4.000000
75%         7.000000
max         9.000000
Name: Facies, dtype: float64

In [3]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(feature_vectors)
scaled_features = scaler.transform(feature_vectors) #ndarray now. 

In [4]:
from sklearn.cross_validation import train_test_split
X_train, X_cv, y_train, y_cv = train_test_split(scaled_features, facies_labels,test_size=0.5, random_state=42)
X_train

C:\Users\jchen\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


array([[ 0.78163021,  0.97098416, -0.44696904, ...,  0.11444223,
         0.95329909, -1.47177523],
       [-1.1921925 ,  2.08932582,  0.14767802, ...,  2.23087371,
         0.95329909, -0.29590661],
       [-0.27367186, -1.13130049, -0.54607688, ..., -1.01312802,
        -1.04898873, -0.8386152 ],
       ...,
       [ 0.2221964 ,  0.04857211,  0.98018391, ..., -0.74689616,
        -1.04898873,  0.94954002],
       [ 0.53236719, -0.39031519, -0.20911021, ..., -0.96726455,
        -1.04898873,  0.72341145],
       [ 0.34268302, -0.51115421,  1.13875646, ..., -0.78940377,
        -1.04898873, -1.33261918]])

In [5]:
from sklearn.metrics import classification_report
target_names = ['SS', 'CSiS', 'FSiS', 'SiSh','MS', 'WS', 'D','PS', 'BS']
y_test = test_well['Facies']
well_features = test_well.drop(['Facies','Formation','Well Name','Depth'],axis=1)
X_test = scaler.transform(well_features)

In [6]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
clf = BaggingClassifier(DecisionTreeClassifier(max_depth=4,min_samples_leaf=10),n_estimators=300,max_samples=1000,
                        bootstrap=True, n_jobs=-1,oob_score=True)
clf.fit(X_train, y_train)
clf.oob_score_

0.5650611071171819

In [7]:
y_cv_pred = clf.predict(X_cv) 
print(classification_report(y_cv, y_cv_pred,target_names=target_names))

             precision    recall  f1-score   support

         SS       0.90      0.59      0.71        92
       CSiS       0.66      0.82      0.74       331
       FSiS       0.70      0.54      0.61       230
       SiSh       0.44      0.67      0.53        82
         MS       0.00      0.00      0.00       111
         WS       0.45      0.75      0.56       202
          D       0.62      0.13      0.22        38
         PS       0.60      0.49      0.54       227
         BS       0.57      0.63      0.60        79

avg / total       0.57      0.59      0.56      1392



C:\Users\jchen\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [8]:
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred,target_names=target_names))

             precision    recall  f1-score   support

         SS       0.20      0.02      0.04        89
       CSiS       0.31      0.79      0.45        89
       FSiS       0.74      0.39      0.51       117
       SiSh       0.20      0.57      0.30         7
         MS       0.00      0.00      0.00        19
         WS       0.68      0.73      0.70        71
          D       1.00      0.06      0.11        17
         PS       0.45      0.62      0.53        40

avg / total       0.48      0.45      0.40       449



C:\Users\jchen\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1428: UserWarning: labels size, 8, does not match size of target_names, 9
  .format(len(labels), len(target_names))
C:\Users\jchen\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
